In [15]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import folium

# First we make our first request to foursquare to get venues from Santiago, Chile

In [16]:
CLIENT_ID='CLIENT_SECRET='VERSION = '20180605' # Foursquare API version
LIMIT=300
lat=-33.4372
lng=-70.6506
radius=5000

In [17]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
results=requests.get(url).json()

In [18]:
import json

In [19]:
venues=[] #pd.DataFrame(columns=['Id','Name','Address','Latitude','Longitude','Category'])
res=results['response']['groups'][0]['items']
for r in res: 
    address='NA'
    try:
        address=r['venue']['location']['address']
    except:
        pass
    venues.append([(r['venue']['id'],
                   r['venue']['name'],
                   address,
                   r['venue']['location']['lat'],
                   r['venue']['location']['lng'],
                   r['venue']['categories'][0]['name'])])
#venues.columns=['Id','Name','Address','Latitude','Longitude','Category']

In [20]:
venuedf=pd.DataFrame([item for venue in venues for item in venue])
venuedf.columns=['Id','Name','Address','Latitude','Longitude','Category']

# Santiago Map

In [22]:
stgo_map=folium.Map(location=[lat,lng],zoom_start=13)
stgo_map

In [8]:
#Get number of venues
venuedf.shape[0]

100

## Foursquare limits us to 100 results per query so we will subdivide the city in its "Comunas" and do a query for each one of them

The data from the comunas is taken from an Arcgis shape file downloaded from https://www.bcn.cl/obtienearchivo?id=repositorio/10221/10396/2/Comunas.zip.  

To work with shape files, we need the geopandas library.

In [23]:
import geopandas as gpd

In [24]:
comunas=gpd.read_file('comunas.shp')

In [25]:
comunas=comunas[comunas['Region'].str.contains('Metropolitana')]

Data is in UTM coordinates, so we need to convert it to latitude and longitude coordinates reference system

In [26]:
comunas=comunas.to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
comunas['latitude']=comunas['geometry'].apply(lambda geo: geo.centroid.y) # We take the centroids of each comuna
comunas['longitude']=comunas['geometry'].apply(lambda geo: geo.centroid.x)

/home/enrique/anaconda3/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [27]:
#Plot comuna's centroids
for lat,lon,comuna,radio in zip(comunas['latitude'],comunas['longitude'],comunas['Comuna'],comunas['shape_leng']/9):
    label=folium.Popup(comuna,parse_html=False)
    color='blue'
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(stgo_map)

In [28]:
stgo_map

In [29]:
# Using a variation of the function from the "Segmenting and clustering in Toronto", we get the venues from each comuna
def getVenues(names, latitudes, longitudes, radius): #Function to get nearby venues for each location
    venues_list=[]
    for name, lat, lng,rad in zip(names, latitudes, longitudes, radius):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            rad, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Comuna', 
                  'Comuna Latitude', 
                  'Comuna Longitude',
                'Venue Id',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
venues=getVenues(comunas['Comuna'],comunas['latitude'],comunas['longitude'],comunas['shape_leng']/10)


San Joaquín
San Miguel
San Ramón
Independencia
La Cisterna
Peñalolén
Providencia
La Reina
Calera de Tango
Colina
Santiago
Lampa
Pirque
Puente Alto
Huechuraba
San Bernardo
Curacaví
María Pinto
Cerrillos
Cerro Navia
Vitacura
Conchalí
El Bosque
Estación Central
La Florida
La Granja
La Pintana
Las Condes
Lo Barnechea
Lo Espejo
Lo Prado
Macul
Maipú
Ñuñoa
Pedro Aguirre Cerda
Pudahuel
Quilicura
Quinta Normal
Recoleta
Renca
El Monte
Padre Hurtado
Peñaflor
Talagante
Paine
Isla de Maipo
Buin
San José de Maipo
Tiltil
Melipilla
San Pedro
Alhué


In [31]:
#We save and load our variables so we don't have to make requests to foursquare every time 
import pickle
#with open('venues_raw.p','rb') as f:
 #   venues=pickle.load(f)
with open('venues_raw.p','wb') as p:
    pickle.dump(venues,p)

Get distance to comuna's centroid

In [34]:
from geopy import distance
def getDistance(x1,y1,x2,y2):
    point1=(x1,y1)
    point2=(x2,y2)
    return distance.distance(point1,point2)
venues['Distance to Comuna']=venues.apply(lambda row: getDistance(row['Venue Longitude'],row['Venue Latitude'],row['Comuna Longitude'],row['Comuna Latitude']),axis=1)

There are venues belonging to 2 comunas so we assign them to the closest centroid. It will still have some errors that we will correct (not completely) by assigning those venues that have a Comuna name in its Venue name, to its corresponding Comuna

In [35]:
venues.sort_values(by='Distance to Comuna',inplace=True)
venues.drop_duplicates(subset='Venue Id',keep='first',inplace=True)
for comuna in venues['Comuna'].unique():
    venues['Comuna'][venues['Venue'].str.contains(comuna)]=comuna

/home/enrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [36]:
venues['Comuna'][venues['Venue'].str.contains(u'Peñalolen')]=u'Peñalolén' #Fixing some typos

/home/enrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Now we get popularity for each venue from venues ratings

In [39]:
ratings=[]
for venue_ID in venues['Venue Id']:
    urllikes='https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v=20191010'.format(venue_ID,CLIENT_ID,CLIENT_SECRET)
    likes=requests.get(urllikes).json()
    try:
        ratings.append(likes['response']['likes']['count'])
    except:
        ratings.append(np.nan)
        pass

In [40]:
#Saving and loading variables again so we don't have to make requests every time 

with open('ratings.p','wb') as r:
    pickle.dump(ratings,r)
#with open('ratings.p','rb') as f:
 #   ratings=pickle.load(f)

In [41]:
venues['Ratings number']=ratings

In [42]:
venues.head()

,Comuna,Comuna Latitude,Comuna Longitude,Venue Id,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance to Comuna,Ratings number
562,Santiago,-33.453531,-70.656552,540b601b498e9d0f44c496c2,Aderezos,-33.454418,-70.656437,Peruvian Restaurant,0.03522136637927094 km,13
1544,Macul,-33.489414,-70.599990,4e455fa0b0fb93df26faa004,Tio Rico,-33.490184,-70.600272,Restaurant,0.04253653465090865 km,15
166,La Cisterna,-33.530081,-70.663889,4f079a5ae4b05dd3a24a72e2,Nagai Sushi,-33.531266,-70.663718,Sushi Restaurant,0.04779718273693551 km,55
1567,Macul,-33.489414,-70.599990,4cb4f42156fca1cd078e4b18,Farmacias Cruz Verde,-33.490775,-70.599892,Pharmacy,0.051634775293416375 km,0
1327,La Pintana,-33.587339,-70.636860,4c4c9427bad7a593b03975ad,Estadio Municipal de La Pintana,-33.586488,-70.636437,Soccer Stadium,0.056735066229530555 km,33


In [43]:
venues=venues.sort_values(by='Ratings number',ascending=False)

# Exploratory Data Analysis

### Comunas with most venues

In [44]:
venues.groupby('Comuna').count().sort_values(by='Venue',ascending=False)

,Comuna Latitude,Comuna Longitude,Venue Id,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance to Comuna,Ratings number
Comuna,,,,,,,,,
Santiago,113,113,113,113,113,113,113,113,113
Ñuñoa,99,99,99,99,99,99,99,99,99
Providencia,99,99,99,99,99,99,99,99,99
Maipú,98,98,98,98,98,98,98,98,98
Puente Alto,96,96,96,96,96,96,96,96,96
Vitacura,96,96,96,96,96,96,96,96,96
La Reina,93,93,93,93,93,93,93,93,93
San Bernardo,81,81,81,81,81,81,81,81,81
La Florida,80,80,80,80,80,80,80,80,80


### Analyze each Comuna

In [45]:

stgo_onehot=pd.get_dummies(venues[['Venue Category']],prefix="",prefix_sep="")

In [46]:

stgo_onehot.head()#=stgo_onehot[['Comuna']+list(stgo_onehot.drop('Comuna',axis=1).columns)]

,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Water Park,Waterfall,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
1824,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1729,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
568,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
stgo_onehot['Comuna']=venues['Comuna']

In [48]:
stgo_onehot=stgo_onehot[['Comuna']+list(stgo_onehot.drop('Comuna',axis=1).columns)]

In [49]:
stgo_grouped=stgo_onehot.groupby('Comuna').mean().reset_index()

## Get most popular comunas

In [50]:
venues_grouped=venues.groupby('Comuna')[['Ratings number']].sum()
venues_grouped.sort_values(by='Ratings number',ascending=False).head(10)

,Ratings number
Comuna,
Providencia,20272
Santiago,18429
Ñuñoa,12700
Pudahuel,9340
Vitacura,8961
Las Condes,5471
San José de Maipo,3767
La Florida,3497
La Reina,3490


## Create function from lab to return most common venues for each neighborhood

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Comuna']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
comuna_venues_sorted = pd.DataFrame(columns=columns)
comuna_venues_sorted['Comuna'] = stgo_grouped['Comuna']

for ind in np.arange(stgo_grouped.shape[0]):
    comuna_venues_sorted.iloc[ind, 1:] = return_most_common_venues(stgo_grouped.iloc[ind, :], num_top_venues)

comuna_venues_sorted.head()

,Comuna,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Alhué,Restaurant,Concert Hall,History Museum,Market,Plaza,Field,Falafel Restaurant,Farm
1,Buin,Restaurant,Sushi Restaurant,Italian Restaurant,Vineyard,German Restaurant,Café,City,BBQ Joint
2,Calera de Tango,Restaurant,Latin American Restaurant,Café,Park,Burger Joint,Gym,Diner,South American Restaurant
3,Cerrillos,Fast Food Restaurant,Department Store,Pharmacy,Sandwich Place,Ice Cream Shop,Clothing Store,Burger Joint,Café
4,Cerro Navia,Plaza,Liquor Store,Japanese Restaurant,Pharmacy,Burger Joint,Mountain,Flea Market,Café


# Get most popular comunas

In [53]:
comuna_venues_sorted.set_index('Comuna',inplace=True)
comuna_venues_sorted['Popularity']=venues_grouped['Ratings number']

In [54]:
comuna_venues_sorted.sort_values(by='Popularity',ascending=False).head(10)

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,Popularity
Comuna,,,,,,,,,
Providencia,Sandwich Place,Restaurant,Bar,Pizza Place,French Restaurant,Café,Plaza,Vegetarian / Vegan Restaurant,20272
Santiago,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Plaza,Hotel,Peruvian Restaurant,18429
Ñuñoa,Bakery,Italian Restaurant,Restaurant,Café,Bar,Gym,Peruvian Restaurant,Pet Store,12700
Pudahuel,Airport Lounge,Coffee Shop,Hotel,Sushi Restaurant,Soccer Field,Breakfast Spot,Airport,Airport Terminal,9340
Vitacura,Café,Sushi Restaurant,Gym / Fitness Center,Italian Restaurant,Bakery,Restaurant,Spa,Park,8961
Las Condes,Bakery,Coffee Shop,Supermarket,Fast Food Restaurant,Multiplex,Plaza,Restaurant,Soccer Stadium,5471
San José de Maipo,Scenic Lookout,Mountain,Restaurant,Hotel,Campground,Breakfast Spot,Other Great Outdoors,Diner,3767
La Florida,Sushi Restaurant,Chinese Restaurant,Pizza Place,Gym,Soccer Stadium,Restaurant,Snack Place,Bakery,3497
La Reina,Restaurant,Sushi Restaurant,Soccer Field,Chinese Restaurant,Pizza Place,Bar,Pet Store,Pharmacy,3490


# Create algorithm to make day plans minimizing distance traveled and maximizing popularity

In [55]:
from scipy.optimize import minimize
from scipy.spatial import distance_matrix
venuesToVisit=['Stadium','Sushi Restaurant','Park','Art Museum'] #Say during a day we want to visit a popular venue of each category
venuesForDay=venues[venues['Venue Category'].isin(venuesToVisit)] #Filter all venues to fit venues we want to visit
venuesForDay=venuesForDay.sort_values(by=['Venue Category','Ratings number'],ascending=False)
venuesForDay=venuesForDay.groupby('Venue Category').head(1) #Get most popular venue for each category

In [56]:
venuesForDay.reset_index(drop=True,inplace=True)

# Optimize distance traveled

To optimize our routes, we need to solve the following linear programming problem

In [57]:
%%latex
$$min\; (\sum\limits_{j=0}^{n-1}\sum\limits_{i=0}^{n-1}C_{i,j}*D_{i,j})$$
where
$$C_{i,j}$$ is 1 if we travel from i to j, 0 otherwise
and 
$$D_{i,j}$$ is the distance between venue i and j

$$s.t$$
Can't go to same venue
$$C_{i,i}=0\;\;\forall{i}\in venues$$
Have to visit each venue once
$$\sum\limits_{k=0,k\neq i}^{n-1}C_{i,k}=1\;\;\forall{k}\in venues$$
Can only go to one venue from each venue
$$\sum\limits_{i=0,i\neq k}^{n-1}C_{i,k}=1\;\;\forall{k}\in venues$$

Avoid sub-tours
$$\sum\limits_{i,j\in S}x_{i,j}<=|S|-1\;\;\forall{S} \subset venues, S \neq \emptyset$$

<IPython.core.display.Latex object>

In [58]:
dist_matrix=(np.matrix(distance_matrix(venuesForDay[['Venue Latitude','Venue Longitude']],venuesForDay[['Venue Latitude','Venue Longitude']]))) #We generate distance matrix for each venue to visit

Next cell is to find all subsets within our venues to visit so we can program the last constraint

In [59]:
import itertools
from operator import or_
from functools import reduce # python3 required

def findsubsets(S,m):
    return list(itertools.combinations(S, m))

n=dist_matrix.shape[0]
allsubsets=[]
for i in range(2,n):
    allsubsets+=findsubsets(range(n),i)

#allsubsets=reduce(or_, allsubsets)

In [60]:
allroutes=[]
for subset in allsubsets:
    allroutes.append(list(itertools.permutations(subset,2)))

## Programming the Linear Optimization Problem using the library pulp

In [63]:
import pulp as p 
  
# Create a LP Minimization problem 
Lp_prob = p.LpProblem('Problem', p.LpMinimize)  
indices=[(x,y) for x in np.arange(dist_matrix.shape[0]) for y in np.arange(dist_matrix.shape[1])]

# Create problem Variables  
x = p.LpVariable.dicts("C",indices, cat = 'Binary')   # Create a variable x >= 0 
u = p.LpVariable.dicts("u_",range(n),cat='Integer') #Create auxiliary variable

# Objective Function 
Lp_prob += p.lpSum(x[i]*dist_matrix[i] for i in indices)
  #Seguir con las constraints!
# Constraints: 

for i,j in indices:
    if i==j:
        Lp_prob+=x[(i,j)]==0
ven=set([i for (i,j) in indices])

for _,k in indices:
    Lp_prob+=sum(x[i,k] for i in ven)==1

for _,j in indices:
    Lp_prob+=sum(x[(i,j)] for i in ven)==sum(x[(j,i)] for i in ven)

for route,subset in zip(allroutes,allsubsets):
    Lp_prob+=sum(x[r] for r in route)<=len(subset)-1
#for i in range(n-1):
 #   for j in range(i+1,n):
  #      Lp_prob+=p.lpSum([u[i]-u[j]+n*x[i,j]])<=n-1
    
# Display the problem 
print(Lp_prob)

status = Lp_prob.solve()   # Solver 
print(p.LpStatus[status])   # The solution status
  
 

Problem:
MINIMIZE
0.11856448088847438*C_(0,_1) + 0.045927281937978216*C_(0,_2) + 0.10081833991293673*C_(0,_3) + 0.11856448088847438*C_(1,_0) + 0.11872662466706646*C_(1,_2) + 0.02074646807240989*C_(1,_3) + 0.045927281937978216*C_(2,_0) + 0.11872662466706646*C_(2,_1) + 0.10683560648186449*C_(2,_3) + 0.10081833991293673*C_(3,_0) + 0.02074646807240989*C_(3,_1) + 0.10683560648186449*C_(3,_2) + 0.0
SUBJECT TO
_C1: C_(0,_0) = 0

_C2: C_(1,_1) = 0

_C3: C_(2,_2) = 0

_C4: C_(3,_3) = 0

_C5: C_(0,_0) + C_(1,_0) + C_(2,_0) + C_(3,_0) = 1

_C6: C_(0,_1) + C_(1,_1) + C_(2,_1) + C_(3,_1) = 1

_C7: C_(0,_2) + C_(1,_2) + C_(2,_2) + C_(3,_2) = 1

_C8: C_(0,_3) + C_(1,_3) + C_(2,_3) + C_(3,_3) = 1

_C9: C_(0,_0) + C_(1,_0) + C_(2,_0) + C_(3,_0) = 1

_C10: C_(0,_1) + C_(1,_1) + C_(2,_1) + C_(3,_1) = 1

_C11: C_(0,_2) + C_(1,_2) + C_(2,_2) + C_(3,_2) = 1

_C12: C_(0,_3) + C_(1,_3) + C_(2,_3) + C_(3,_3) = 1

_C13: C_(0,_0) + C_(1,_0) + C_(2,_0) + C_(3,_0) = 1

_C14: C_(0,_1) + C_(1,_1) + C_(2,_1) + C_(3,_

In [64]:
# Printing the final solution
for k in indices:
    print(k,p.value(x[k]))
        
    

(0, 0) 0.0
(0, 1) 0.0
(0, 2) 0.0
(0, 3) 1.0
(1, 0) 0.0
(1, 1) 0.0
(1, 2) 1.0
(1, 3) 0.0
(2, 0) 1.0
(2, 1) 0.0
(2, 2) 0.0
(2, 3) 0.0
(3, 0) 0.0
(3, 1) 1.0
(3, 2) 0.0
(3, 3) 0.0


## We've got our solution, now we need to display it in a friendly way

First, we take delete the longest route since the algorithm gives us a closed circuit

In [65]:
routes=[k for k in indices if p.value(x[k])==1] #Get routes we are going to take
longestroute=(np.nan,np.nan) #Aux variable to store longest route
longestdistance=0

In [66]:
#Get longest route
for route in routes:
    if dist_matrix[route]>longestdistance:
        longestroute=route
        longestdistance=dist_matrix[route]

In [67]:
routes.remove(longestroute)

## Now, how do we order routes so they can be in logical order?

Here, I use a pandas dataframe and filter it on each step to finally print the solution

In [68]:
dfroutes=pd.DataFrame(routes)

In [69]:
firstroute=dfroutes[dfroutes[0].isin(dfroutes[1])==False]
print("Start in "+str(venuesForDay['Venue'][firstroute.iloc[0,0]]+" in "+str(venuesForDay['Comuna'][firstroute.iloc[0,0]])))
print("Then go to "+str(venuesForDay['Venue'][firstroute.iloc[0,1]]+" in "+str(venuesForDay['Comuna'][firstroute.iloc[0,1]])))
lastvenue=firstroute.iloc[0,1]
for route in range(1,dfroutes.shape[0]):
    nextroute=dfroutes[dfroutes[0]==lastvenue]
    print("Then go to %s in %s"%(str(venuesForDay['Venue'][nextroute.iloc[0,1]]),str(venuesForDay['Comuna'][nextroute.iloc[0,1]])))
    lastvenue=nextroute.iloc[0,1]

Start in Parque Padre Hurtado in Padre Hurtado
Then go to Niu Sushi in Vitacura
Then go to Centro Cultural Palacio La Moneda in Santiago
Then go to Movistar Arena in Santiago


# Done :)